In [ ]:
# %pip install pandas

  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------ --------------------------------- 1.8/11.3 MB 11.2 MB/s eta 0:00:01
   ---------------- ----------------------- 4.7/11.3 MB 13.0 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.3 MB 16.3 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 15.4 MB/s eta 0:00:00
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/4 [pytz]
   ---------------------------------------- 0/4 [pytz]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- ----------------------------- 1/4 [tzdata]
   ---------- --

In [11]:
import os
import json
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\vishn\AppData\Roaming\nltk_data...


True

In [12]:
import os
import pandas as pd

data_dir = "data/train"
train_csv = "data/train.csv"

# Load train.csv
df = pd.read_csv(train_csv)

# Detect correct column names
article_id_col = "article_id" if "article_id" in df.columns else df.columns[0]
real_col = "real" if "real" in df.columns else df.columns[-1]

# Prepare list for paired texts
paired_data = []

for idx, row in df.iterrows():
    article_id = f"article_{int(row[article_id_col]):04d}"
    file_1_path = os.path.join(data_dir, article_id, "file_1.txt")
    file_2_path = os.path.join(data_dir, article_id, "file_2.txt")
    try:
        with open(file_1_path, "r", encoding="utf-8") as f1:
            text_1 = f1.read()
    except Exception:
        text_1 = ""
    try:
        with open(file_2_path, "r", encoding="utf-8") as f2:
            text_2 = f2.read()
    except Exception:
        text_2 = ""
    paired_data.append({
        "text_1": text_1,
        "text_2": text_2,
        "real": row[real_col]  # 1 or 2
    })

paired_df = pd.DataFrame(paired_data)
paired_df.head()

,text_1,text_2,real
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...,1
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...,2
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...,1
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...,2
4,Dinosaur Rex was excited about his new toy set...,Analyzing how fast stars rotate within a galax...,2


In [13]:
def clean_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    
    # Join the tokens back into a cleaned string
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text



paired_df['cleaned_text_1'] = paired_df['text_1'].apply(clean_text)
paired_df['cleaned_text_2'] = paired_df['text_2'].apply(clean_text)
paired_df.head()

,text_1,text_2,real,cleaned_text_1,cleaned_text_2
0,The VIRSA (Visible Infrared Survey Telescope A...,The China relay network has released a signifi...,1,virsa visible infrared survey telescope array ...,china relay network released significant amoun...
1,China\nThe goal of this project involves achie...,The project aims to achieve an accuracy level ...,2,china goal project involves achieving accuracy...,project aim achieve accuracy level dex analyzi...
2,Scientists can learn about how galaxies form a...,Dinosaur eggshells offer clues about what dino...,1,scientist learn galaxy form evolve two method ...,dinosaur eggshell offer clue dinosaur ate long...
3,China\nThe study suggests that multiple star s...,The importance for understanding how stars evo...,2,china study suggests multiple star system play...,importance understanding star evolve led resea...
4,Dinosaur Rex was excited about his new toy set...,Analyzing how fast stars rotate within a galax...,2,dinosaur rex excited new toy set many dinosaur...,analyzing fast star rotate within galaxy compa...
